In [2]:
import csv
import cv2
import numpy as np

lines=[]

with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [3]:
images = []
measurements = []
# for line in lines:
#     source_path = line[0]
#     filename = source_path.split('/')[-1]
#     current_path = './data/IMG/' + filename
#     image = cv2.imread(current_path)
#     images.append(image)
    
#     measurement = float(line[3])
#     measurements.append(measurement)

In [4]:
import pandas as pd
log_pd = pd.read_csv('data/driving_log.csv')



In [5]:
measurements = log_pd['steering']
image_fname = log_pd['center']

In [6]:
print(measurements.shape)
print(image_fname.shape)
print(measurements[0:2])
print(image_fname[0:2])
print(type(measurements))
print(type(image_fname))
measurements=np.array(measurements)
print(type(measurements))


(8036,)
(8036,)
0    0.0
1    0.0
Name: steering, dtype: float64
0    IMG/center_2016_12_01_13_30_48_287.jpg
1    IMG/center_2016_12_01_13_30_48_404.jpg
Name: center, dtype: object
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [ ]:
# X_train = np.array(images)
# y_train = np.array(measurements)
images = []
for img in image_fname:
    image = cv2.imread('data/' + img)
    images.append(image)
    images.append(cv2.flip(image,1))
y_train =[]    
for meas in measurements:
    y_train.append(meas)
    y_train.append(meas*-1.0)

y_train = np.array(y_train)
X_train = np.array(images)

print(X_train.shape)    
print(y_train.shape)  

In [7]:
for img in image_fname:
    image = cv2.imread('data/' + img)
    images.append(image)


In [10]:
X_train = np.array(images)

In [13]:
temp_x = np.array(images)
X_train = np.append(temp_x,np.fliplr(temp_x),axis=0)
temp_y = np.array(measurements)
y_train = np.append(temp_y,-1.0*temp_y)

In [15]:
print(y_train.shape)
print(X_train.shape)

(16072,)
(16072, 160, 320, 3)


In [16]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Lambda

model = Sequential()
# model.add(Flatten(input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6, 5, 5, activation = 'relu', input_shape=(160,320,3)))

model.add(MaxPooling2D())

model.add(Convolution2D(16, 5, 5, activation = 'relu'))

model.add(MaxPooling2D())
# model.add(Convolution2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train,y_train, validation_split=0.2,shuffle=True, nb_epoch=3)

model.save('model_lenet_aug.h5')

Using TensorFlow backend.


Train on 12857 samples, validate on 3215 samples
Epoch 1/3
12857/12857 [==============================] - 603s - loss: 0.7385 - val_loss: 0.0148